In [ ]:
import dataclasses
import numpy as np
from players.base import play_game
from players.mcts import SearchParameters, PlayerMCTS
from players.strategy import StrategyTokenProducer, Strategy

np.random.seed(3)

mcts_params = SearchParameters(
    num_simulations=100
)
from network.checkpoints import Checkpoint
ckpt = Checkpoint.from_json_file("./data/checkpoints/tr-st/4.json")

st1 = Strategy(table=Strategy.create_empty_table())
st1.table[1, :, :] = 1

st2 = Strategy(table=Strategy.create_empty_table())
st2.table[1, :, :] = 0

player_mcts = PlayerMCTS(
    ckpt.params,
    ckpt.model.create_caching_model(),
    mcts_params
)

strategy = np.zeros((2, 4, 4, 2, 2), dtype=np.uint8)
win_count = np.zeros((7), dtype=np.uint8)

for i in range(1000):
    result = play_game(
        player1=dataclasses.replace(player_mcts, strategy=st1),
        player2=dataclasses.replace(player_mcts, strategy=st2),
        print_board=False,
        token_producer=StrategyTokenProducer()
    )

    win_count[int(result.win_type.value) * result.winner + 3] += 1

    strategy[0] += StrategyTokenProducer.create_strategy_table(result.tokens1)
    strategy[1] += StrategyTokenProducer.create_strategy_table(result.tokens2)

    print(i, win_count)
    print(strategy[0, :, :, 1, 0].sum(), strategy[0, :, :, 1, 1].sum())
    print(strategy[1, :, :, 1, 0].sum(), strategy[1, :, :, 1, 1].sum())


In [2]:
import players.mcts as mcts
import players.strategy as strategy
import players.config as config

cfg = mcts.PlayerMCTSConfig(
    base_name="tr-st",
    step=4,
    mcts_params=config.SearchParametersRange(num_simulations=config.IntRange(100, 1000)),
    strategy_factory=strategy.Random(p=(0.1, 0.1, 0.8))
)


d = {}

d[cfg] = 1

print(d)

{PlayerMCTSConfig(base_name='tr-st', step=4, mcts_params=SearchParametersRange(num_simulations=IntRange(min=100, max=1000), dirichlet_alpha=FloatRange(min=0.3, max=0.3), dirichlet_eps=FloatRange(min=0.25, max=0.25), n_ply_to_apply_noise=IntRange(min=20, max=20), max_duplicates=IntRange(min=3, max=3), c_init=FloatRange(min=1.25, max=1.25), c_base=IntRange(min=25, max=25), depth_search_checkmate_root=IntRange(min=7, max=7), depth_search_checkmate_leaf=IntRange(min=4, max=4), test_c=False, visibilize_node_graph=False), strategy_factory=Random(p=(0.1, 0.1, 0.8))): 1}


In [1]:
from distributed.config import RunConfig

c1 = RunConfig.from_json_file('./data/run_config.json')
s = c1.to_json()
c2 = RunConfig.from_json(s)

print(c2)

RunConfig(project_name='geister-rmt', wandb_log=False, series_length=4, tokens_length=220, update_period=16, replay_buffer_size=40000, init_replay_buffer=None, agent=AgentConfig(init_params=FromCheckpoint(dir_name='./data/checkpoints/tr-st', step=4), training=TrainingConfig(batch_size=32, num_batches=16, learning_rate=0.005), match_making=MatchMakingConfig(mathod=PFSP(p=4.0), buffer_size=2048), condition_for_keeping_snapshots=ConditionForKeepingSnapshots(win_rate_threshold=None, step_period=50), mcts_params=SearchParametersRange(num_simulations=IntRange(min=10, max=40), dirichlet_alpha=FloatRange(min=0.1, max=0.4), dirichlet_eps=FloatRange(min=0.25, max=0.25), n_ply_to_apply_noise=IntRange(min=10, max=20), max_duplicates=IntRange(min=1, max=8), c_init=FloatRange(min=1.25, max=1.25), c_base=IntRange(min=10, max=50), depth_search_checkmate_root=IntRange(min=6, max=8), depth_search_checkmate_leaf=IntRange(min=3, max=5), test_c=False, visibilize_node_graph=False)), project_dir='./data/proj